In [ ]:
from astra import Astra

import matplotlib.pyplot as plt
import matplotlib
import os
import numpy as np
import CMethods
import scipy.constants as const

matplotlib.rcParams['figure.figsize'] = (12,8)
%config InlineBackend.figure_format = 'retina'

A0 = Astra("astra.in")

b_field = 10
A0.input['solenoid']['MaxB(1)'] = b_field
A0.input['newrun']['zstop'] = 0.6
A0.input['aperture']['lapert'] = False
A0.input['aperture']['Ap_Z1(1)'] = 0.04
A0.input['aperture']['Ap_Z2(1)'] = 0.34
A0.timeout = None
A0.verbose = True

#Run
A0.run()
A0.plot()
A0.plot(['norm_emit_x', 'norm_emit_y'], xlim = (0, 10), figsize=(15,8) )


In [ ]:


output_particles = A0.output["particles"][-1]

A1 = Astra("astra.in")

b_field = 5


A1.input['solenoid']['MaxB(1)'] = b_field
A1.input['solenoid']['S_pos(1)'] = 2

A1.input['aperture']['lapert'] = False
A1.input['aperture']['Ap_Z1(1)'] = 2.04
A1.input['aperture']['Ap_Z2(1)'] = 2.34

A1.verbose = True
A1.track(output_particles,2.5)

particles = A1.output["particles"][-1]

print(particles["x"])
print(particles["mean_y"])

A1.plot()



In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cmap = matplotlib.colormaps.get_cmap('inferno')

skip=1
X = np.array([P0.x for P0 in  A0.particles]).T[::skip]
Y = np.array([P0.y for P0 in  A0.particles]).T[::skip]
Z = np.array([P0.z for P0 in  A0.particles]).T[::skip]

scale = np.hypot(X[:, 0], Y[:,0]).max()

# color by  initial radius
for i in range(len(X)):
    color = cmap(1-np.hypot(X[i,0], Y[i,0])/scale)
    ax.plot(X[i]*1e3, Y[i]*1e3, Z[i], zdir='x', color=color, linewidth=.3)

ax.set_box_aspect((2,1,1))    
    
ax.set_xlabel('Z (m)')
ax.set_ylabel('X (mm)')
ax.set_zlabel('Y (mm)')

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cmap = matplotlib.colormaps.get_cmap('inferno')

skip=1
X = np.array([P0.x for P0 in  A1.particles]).T[::skip]
Y = np.array([P0.y for P0 in  A1.particles]).T[::skip]
Z = np.array([P0.z for P0 in  A1.particles]).T[::skip]

scale = np.hypot(X[:, 0], Y[:,0]).max()

# color by  initial radius
for i in range(len(X)):
    color = cmap(1-np.hypot(X[i,0], Y[i,0])/scale)
    ax.plot(X[i]*1e3, Y[i]*1e3, Z[i], zdir='x', color=color, linewidth=.3)

ax.set_box_aspect((2,1,1))    
    
ax.set_xlabel('Z (m)')
ax.set_ylabel('X (mm)')
ax.set_zlabel('Y (mm)')

In [ ]:
from astra import Astra
import torch
import numpy as np

def toy_model(b_0, b_1, d_0, d_1):
    
    print(b_0, b_1, d_0, d_1)

    z_end = 5.4                                 #end position of simulation, e.g. diagnostics
    z_0 = 0.04                                  #position of the first element

    l_sol = 0.3                                 #solenoid length
    d_buffer = 0.2                              #buffer between elements

    z_seg_0 = z_0 + d_0 + l_sol + d_buffer      #length of the first simulation segment
    z_seg_1 = d_1 + l_sol
    z_seg_12 = z_seg_0 + z_seg_1
    z_rem = z_end - z_seg_12

    #Configure first solenoid

    A0 = Astra("astra.in")

    A0.input['aperture']['lapert'] = False
    A0.input['aperture']['Ap_Z1(1)'] = z_0+d_0
    A0.input['aperture']['Ap_Z2(1)'] = z_0+d_0+l_sol

    A0.input['solenoid']['MaxB(1)'] = b_0

    A0.input['newrun']['zstart'] = 0
    A0.input['newrun']['zstop'] = z_seg_0
    output_particles_0 = None

    try:
        if z_rem > 0:
            A0.run()
            output_particles_0 = A0.output["particles"][-1]
        else:
            sigma_x = 0
            sigma_y = 0
            dx = 0
            dy = 0

    except ValueError:
        sigma_x = 0
        sigma_y = 0
        dx = 0
        dy = 0

    A1 = Astra("astra.in")
    A1.input['aperture']['lapert'] = False
    A1.input['aperture']['Ap_Z1(1)'] = z_seg_0 + d_1
    A1.input['aperture']['Ap_Z2(1)'] = z_seg_0 + d_1 + l_sol

    A1.input['solenoid']['MaxB(1)'] = b_1

    A1.input['newrun']['zstart'] = z_seg_0
    A1.input['newrun']['zstop'] = z_end

    try:
        if z_rem > 0:
            A1.track(output_particles_0,2.5)
            output_particles_1 = A1.output["particles"][-1]
            sigma_x = output_particles_1["x"]
            sigma_y = output_particles_1["y"]
        else:
            sigma_x = 0
            sigma_y = 0
            dx = 0
            dy = 0

    except ValueError:
        sigma_x = 0
        sigma_y = 0
        dx = 0
        dy = 0
        
    except UnboundLocalError:
        sigma_x = 0
        sigma_y = 0
        dx = 0
        dy = 0
    
    return sigma_x, sigma_y


class AstraOptimizer(torch.nn.Module):

    def __init__(self, params):
        super().__init__()
        # register set of quad strengths as parameter:
        self.register_parameter('params',torch.nn.Parameter(params, requires_grad=True))

    def forward(self):
        # create lattice given quad strengths in k_set:
        sigma = toy_model(self.params[0].item(), self.params[1].item(), self.params[2].item(), self.params[3].item())
        try:
            sigma_x = torch.std(torch.tensor(sigma[0]))
            sigma_y = torch.std(torch.tensor(sigma[1]))
            print(sigma_x, sigma_y)
        except TypeError:
            sigma_x = torch.zeros(1)
            sigma_y = torch.zeros(1)

        sigma_target = 0.0001# calculate and return loss function:
        dx = (sigma_x - sigma_target)
        dy = (sigma_y - sigma_target)
        
        #hier add zu sqrt
        return torch.add(dx ** 2 + dy ** 2)


def train_model(model, training_iter, alpha=0.1):
    history_param = [None] * training_iter  # list to save params
    history_loss = [None] * training_iter  # list to save loss

    # print the trainable parameters
    for param in model.named_parameters():
        print(f'{param[0]} : {param[1]}')

    # Use PyTorch Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), alpha)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Calc loss and backprop gradients
        loss = model()  # loss is just O.F.
        #loss.requires_grad = True
        loss.backward()  # gradient

        # print info:
        if i % 100 == 0:  # print each 100 steps
            print('Iter %d/%d - Loss: %.5f ' % (
                i + 1, training_iter, loss
            ))

        # save loss and param
        for param in model.parameters():
            history_param[i] = param.data.detach().numpy().copy()
        history_loss[i] = loss.detach().numpy().copy()

        # optimization step
        optimizer.step()

    # returns params and loss for every iteration
    return np.asarray(history_param), np.asarray(history_loss)



In [ ]:

k_set = torch.zeros(4)

# Optimization
model = AstraOptimizer(k_set)
params, loss = train_model(model, 1000, 0.01)

In [ ]:
print(torch.backends.mps.is_available())

In [ ]:
k_0 = torch.tensor(3)
k_1 = torch.tensor(2)
k = torch.div(k_0, k_1)
print(torch.sin(torch.mul(k_0,k_1)))


In [1]:
%load_ext autoreload
%autoreload 2
import CMethods
import torch
import numpy as np
test_particles_xyz = CMethods.astra_file_to_ttm("test_plasma_file_protons.part", 10, "proton")

print(test_particles_xyz.size(0))
x = torch.empty(0,6)
y =torch.tensor([1,2,3,4,5,6])
if x.size(0) == 0:
    x = y
else:
    x = torch.stack((x, y),0)

y =torch.tensor([1,2,3,4,5,6])
if x.size(0) == 0:
    x = y
else:
    x = torch.stack((x, y),0)
    


print(test_particles_xyz)

    




531
tensor([[-4.1014e-05, -1.8698e-01, -1.2264e-05, -5.5894e-02,  0.0000e+00,
         -8.0249e-02],
        [-1.8987e-06, -8.6527e-03,  1.2288e-05,  5.5999e-02,  0.0000e+00,
         -8.0210e-02],
        [-8.0363e-06, -3.6624e-02,  4.0598e-05,  1.8508e-01,  0.0000e+00,
         -8.0246e-02],
        ...,
        [ 2.3463e-06,  1.0693e-02, -6.1445e-06, -2.8002e-02,  0.0000e+00,
         -8.0207e-02],
        [-2.5910e-06, -1.1808e-02,  4.2457e-05,  1.9356e-01,  0.0000e+00,
         -8.0248e-02],
        [-6.4207e-06, -2.9261e-02,  1.6722e-05,  7.6210e-02,  0.0000e+00,
         -8.0214e-02]], dtype=torch.float64, requires_grad=True)


In [10]:
import elements

def toy_model(b_0, b_1, d_0, d_1):
    
    print(b_0, b_1, d_0, d_1)
    
    segments = 100

    z_end = 5.4                                 #end position of simulation
    z_0 = 0.04
    
    ref_energy = 10

    l_sol = torch.tensor(0.3, requires_grad=True)                                 #solenoid length
    #d_buffer = 0.2                              #buffer between elements
    
    drift_0 = elements.Drift(segments, z_0, ref_energy)
    drift_1 = elements.Drift(segments, d_0, ref_energy)
    solenoid_0 = elements.Solenoid(segments, b_field=b_0, length=l_sol, ref_energy=ref_energy)
    drift_2 = elements.Drift(segments, d_1, ref_energy)
    solenoid_1 = elements.Solenoid(segments, b_field=b_1, length=l_sol, ref_energy=ref_energy)
    
    z_rem = z_end - (z_0+d_0+2*l_sol+d_1)
    
    drift_3 = elements.Drift(segments, z_rem, ref_energy)
    
    toy_list = [drift_0, drift_1, solenoid_0, drift_2, solenoid_1, drift_3]

    
    beamline_0 = elements.Beamline()


#    if z_rem > 0:
#        [output_parts, rms] = beamline_0.track(element_list=toy_list, input_particles=test_particles_xyz)
#        sigma_x = rms[0][-1].item()
#        sigma_y = rms[1][-1].item()
#    else:
#        sigma_x = 0
#        sigma_y = 0
#        dx = 0
#        dy = 0
    [output_parts, rms] = beamline_0.track(element_list=toy_list, input_particles=test_particles_xyz)
    return rms

class AstraOptimizer(torch.nn.Module):

    def __init__(self, par):
        super().__init__()
        # register set of parameter:
        self.register_parameter('par',torch.nn.Parameter(par, requires_grad=True))

    def forward(self):
        # create lattice given quad strengths in k_set:
        rms = toy_model(self.par[0], self.par[1], self.par[2], self.par[3])
        
        #sigma_x = torch.std(rms[-1][:, 0].clone())
        #sigma_y = torch.std(rms[-1][:, 2].clone())
        sigma_x = torch.std(rms[-1].clone())
        sigma_y = torch.std(rms[-1].clone())

        #sigma_target = torch.tensor(0.0001, dtype=torch.float64)# calculate and return loss function:
        sigma_target = 0.001
        dx = (sigma_x - sigma_target)
        dy = (sigma_y - sigma_target)

        return torch.sqrt(torch.add(torch.square(dx), torch.square(dy)))



def train_model(model, training_iter, alpha=0.1):
    history_param = [None] * training_iter  # list to save params
    history_loss = [None] * training_iter  # list to save loss

    # print the trainable parameters
    for param in model.named_parameters():
        print(f'{param[0]} : {param[1]}')

    # Use PyTorch Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), alpha)

    for i in range(training_iter):
    
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Calc loss and backprop gradients
        #with torch.autograd.detect_anomaly():
        loss = model()  # loss is just O.F.
        loss.backward()  # gradient#
        optimizer.step()
        
        # print info:
        if i % 10 == 0:  # print each 100 steps
            print('Iter %d/%d - Loss: %.5f ' % (
                i + 1, training_iter, loss
            ))

        # save loss and param
        #for param in model.parameters():
        #    history_param[i] = param.data.detach().numpy().copy()
        #history_loss[i] = loss.detach().numpy().copy()

        # optimization step
        

    # returns params and loss for every iteration
    return np.asarray(history_param), np.asarray(history_loss)




In [17]:
k_set = torch.tensor([3, 2, 2, 2], requires_grad=True, dtype=torch.float64)

# Optimization
model = AstraOptimizer(k_set)

params, loss = train_model(model, 1000, 1)

par : Parameter containing:
tensor([3., 2., 2., 2.], dtype=torch.float64, requires_grad=True)
tensor(3., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(2., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(2., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(2., dtype=torch.float64, grad_fn=<SelectBackward0>)
Iter 1/1000 - Loss: 2.57778 
tensor(3., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(2., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(1.0000, dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(1.0000, dtype=torch.float64, grad_fn=<SelectBackward0>)
tensor(3., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(2., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(0.0684, dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(0.0186, dtype=torch.float64, grad_fn=<SelectBackward0>)
tensor(3., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(2., dtype=torch.float64, grad_fn=<SelectBackward0>) tensor(-0.4421, dtype=torch.float64

KeyboardInterrupt: 

In [ ]:
[x,y,z] = elements.toy_model(4.235, 2, 0, 4, test_particles_xyz)

rms_plt_test = [x.clone(), y.clone(), z.clone()]

plt.plot(rms_plt_test[2].detach().clone(), rms_plt_test[0].detach().clone())

In [ ]:
a = torch.tensor([1,2,3,4])
b = torch.tensor([5,6,7,8])

tensor_list = [a, b]

if isinstance(a, torch.Tensor):
    a_list = [a]
else:
    a_list = a


a_list.append(b)


print(a_list)